In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pyprind
import csv
import re

In [2]:
länge = 145

tests = []
pbar = pyprind.ProgBar(länge-1)
for i in range(1, länge):
    url = 'http://www.gamestar.de/index.cfm?pid=646&p='+str(i)
    html = urlopen(url)
    bs = BeautifulSoup(html, 'lxml')
    try:
        games = bs.findAll('div', {'class':'teaserImage belazy'})
        for elem in games:
            try:
                wertung = elem.find('div').find('a', {'class':''}).get_text()
            except:
                wertung = ''
            name = elem.nextSibling.nextSibling.find('h3').get_text().rsplit(' ', 2)[0]
            link = 'http://www.gamestar.de'+elem.find('a', {'class':'linkProductM'})['href']
            try:
                release = elem.nextSibling.nextSibling.find('div', {'class':'genreReleaseDate'}).get_text().split('\n')[2]
            except:
                release = ''
            tests.append([wertung, name, release, link])
    except:pass
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:07:55


In [3]:
df = pd.DataFrame(tests, columns=['Wertung', 'Name', 'Release', 'Link'])

quartale = {'Januar':1, 'Februar':1, 'März':1, 'April':2, 'Mai':2, 'Juni':2, 'Juli':3, 'August':3, 'September':3, 'Oktober':4, 'November':4, 'Dezember':4 }

release_date = []
for elem in df['Release']:
    try:
        day = re.search('([0-9]{2})\.', elem).group(1)
    except:
        day = ''
    try:
        month = re.search('[^\W\d_]+', elem).group(0)
        quartal = quartale[month]
    except:
        month = ''
        try:
            quartal = re.search('([0-9])\.', elem).group(1)
        except:
            quartal = ''
    try:
        year = re.search('[0-9]+$', elem).group(0)
    except:
        year = ''
    release_date.append([day, month, year, quartal])
    
df2 = pd.DataFrame(release_date, columns=['Tag', 'Monat', 'Jahr', 'Quartal'])
game = pd.concat([df, df2], axis=1).drop('Release', axis=1)
game[['Wertung', 'Tag', 'Jahr', 'Quartal']] = game[['Wertung', 'Tag', 'Jahr', 'Quartal']].apply(pd.to_numeric)
game.to_csv('build/Gamestar_Test_Links.csv', index=False, encoding='utf-8')

In [4]:
game = pd.read_csv('build/Gamestar_Test_Links.csv')
game.shape

(1420, 7)

In [7]:
with open('build/Gamestar_Test_Data.csv', 'w', encoding='utf-8') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(['Plattform', 'Kategorie', 'Publisher'])

artikel2 = []
pbar = pyprind.ProgBar(len(game['Link']))
for link in game['Link']:
    try:
        html = urlopen(link)
        bs = BeautifulSoup(html, 'lxml')
        header = bs.find('div', {'class':'productHeadTitle'}).get_text()
    except:pass
    try:
        plattform = re.search('- (.+)', header).group(1)
        if '-' in plattform:
            plattform = plattform.rsplit('- ')[-1]
    except:
        plattform = ''
    try:
        kategorie = re.search('.\\n(.*) \\xa0', header).group(1)
    except:
        kategorie = ''
    try:
        publisher = re.search('Publisher: (.*) \\n', header).group(1)
        if publisher == '-':
            publisher = ''
    except:
        publisher = ''
    
    artikel2.append([plattform, kategorie, publisher])
    with open('build/Gamestar_Test_Data.csv', 'a', encoding='utf-8') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerow([plattform, kategorie, publisher])
    
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:41:56


In [8]:
data = pd.read_csv('build/Gamestar_Test_Data.csv')
game = pd.read_csv('build/Gamestar_Test_Links.csv')
df = pd.concat([game, data], axis=1)
df.to_csv('build/Gamestar_Test_Combined.csv', index=False, encoding='utf-8')
df.shape

(1420, 10)